## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tuktoyaktuk,CA,69.4541,-133.0374,46.40,87,90,10.36,overcast clouds
1,1,Belmonte,BR,-15.8631,-38.8828,78.80,63,15,8.95,few clouds
2,2,Chokurdakh,RU,70.6333,147.9167,40.96,78,100,12.50,overcast clouds
3,3,Kruisfontein,ZA,-34.0033,24.7314,56.88,83,78,5.12,broken clouds
4,4,Norman Wells,CA,65.2820,-126.8329,53.64,87,75,9.22,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Belmonte,BR,-15.8631,-38.8828,78.80,63,15,8.95,few clouds
5,5,Albany,US,42.6001,-73.9662,82.35,46,81,9.78,broken clouds
12,12,Aguimes,ES,27.9054,-15.4461,79.66,73,20,29.93,few clouds
13,13,Sinnamary,GF,5.3833,-52.9500,83.25,74,40,9.71,scattered clouds
19,19,Hambantota,LK,6.1241,81.1185,79.95,76,93,18.48,overcast clouds
23,23,Soure,BR,-0.7167,-48.5233,82.04,77,23,10.98,few clouds
24,24,Jamestown,US,42.0970,-79.2353,77.29,55,20,3.00,few clouds
27,27,Nouadhibou,MR,20.9310,-17.0347,82.38,61,0,10.36,clear sky
35,35,Caravelas,BR,-17.7125,-39.2481,76.32,58,12,7.43,few clouds
40,40,Victoria,HK,22.2855,114.1577,82.96,84,100,7.47,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                276
City                   276
Country                276
Lat                    276
Lng                    276
Max Temp               276
Humidity               276
Cloudiness             276
Wind Speed             276
Current Description    276
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Belmonte,BR,78.80,few clouds,-15.8631,-38.8828,
5,Albany,US,82.35,broken clouds,42.6001,-73.9662,
12,Aguimes,ES,79.66,few clouds,27.9054,-15.4461,
13,Sinnamary,GF,83.25,scattered clouds,5.3833,-52.9500,
19,Hambantota,LK,79.95,overcast clouds,6.1241,81.1185,
23,Soure,BR,82.04,few clouds,-0.7167,-48.5233,
24,Jamestown,US,77.29,few clouds,42.0970,-79.2353,
27,Nouadhibou,MR,82.38,clear sky,20.9310,-17.0347,
35,Caravelas,BR,76.32,few clouds,-17.7125,-39.2481,
40,Victoria,HK,82.96,overcast clouds,22.2855,114.1577,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"    
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")     

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
hotel_df.isnull()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False
12,False,False,False,False,False,False,False
13,False,False,False,False,False,False,False
19,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...
683,False,False,False,False,False,False,False
688,False,False,False,False,False,False,False
690,False,False,False,False,False,False,False
693,False,False,False,False,False,False,False


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis=0, how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Belmonte,BR,78.80,few clouds,-15.8631,-38.8828,Pousada Monte Carmelo O Pão
5,Albany,US,82.35,broken clouds,42.6001,-73.9662,
12,Aguimes,ES,79.66,few clouds,27.9054,-15.4461,Hotel Villa de Aguimes
13,Sinnamary,GF,83.25,scattered clouds,5.3833,-52.9500,ALETHEIA Appartement Sinnamary
19,Hambantota,LK,79.95,overcast clouds,6.1241,81.1185,Bungalow 63
23,Soure,BR,82.04,few clouds,-0.7167,-48.5233,Casarão da Amazônia
24,Jamestown,US,77.29,few clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
27,Nouadhibou,MR,82.38,clear sky,20.9310,-17.0347,El Medina
35,Caravelas,BR,76.32,few clouds,-17.7125,-39.2481,Pousada dos Navegantes
40,Victoria,HK,82.96,overcast clouds,22.2855,114.1577,Mini Hotel Central


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.50)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))